In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats
import more_itertools as mit
import urllib
from scipy import stats

%matplotlib inline  


In [ ]:
def histograma(dataset, numero_bins=50, type_bins='log'):
    """
    Calcula el histograma de una serie de datos y guarda el resultado
    El input es una lista, no sirve meter como input un histograma X-Y
    INPUT:
        dataset (list, array): Datos sobre los que se hacen el histograma
        header (str): String for using in header and in file name
        output_path (str): Donde se guardara el resultado. IF OUTPUT_PATH=FALSE
                            devuelve el histograma y no lo guarda a archivo
        numero_bins (int): Numero de bins que se realizaran
        type_bins ('log', 'lin'): Relacion para generar los bins
        output_name (str): Name for the output file with extension .txt
        header_adicional (str): Aditional header info only for the file header
    OUTPUT:
        Guarda el resultado en output_path
        if output_path == FALSE: Devuelve bin_mean_x, counts_mean
    """

    dataset = np.asarray(dataset)

    # Defining edges depending if log or lin
    if (type_bins == 'log'):
        min_x = np.log10(np.min(dataset[dataset.nonzero()]))
        max_x = np.log10(np.max(dataset))
        bin_edges = np.logspace(min_x, max_x, numero_bins)
        
    elif (type_bins == 'lin'):
        min_x = np.min(dataset[dataset.nonzero()])
        max_x = np.max(dataset)
        bin_edges = np.linspace(min_x, max_x, numero_bins)

    # En que bin cae cada dato
    index = np.digitize(dataset, bin_edges)

    # Frecuencia de datos por bin
    datosPorBin = [len(dataset[index == i]) for i in range(1, len(bin_edges))]

    # Adimensionaliza la frecuencia por la anchura del bin
    counts_mean = datosPorBin/np.diff(bin_edges)
    counts_mean = counts_mean[counts_mean != 0]
    
    # version normalizada
    masatotal = np.sum(datosPorBin)
    counts_norm = counts_mean/masatotal

    # Posicion media del eje x
    bin_mean_x = [dataset[index == i].mean() for i in range(1, len(bin_edges))
                  if len(np.asarray(dataset[index == i])) > 0]

    return(bin_mean_x, counts_mean, counts_norm, bin_edges)

In [ ]:
def binear_datos(lista_x, lista_y, bins=50, log = True):
    """
    Dado una lista de par de puntos (X,Y) binea dando lugar a una lista equivalente
    pero de menor dimension. Sirve para eliminar el ruido, variaciones, etc.
    INPUT:
        lista_x (list, array): Lista de puntos de igual dimension que lista_y
        lista_y (list, array): Lista de puntos de igual dimension que lista_x
        bins (int): numero de bins a utilizar
        log (boolean): True-> bins logaritmico, False-> bins lineales
    OUTPUT:
        posiciones (array): Centro de gravedad del bin. lista de posiciones X
        frecuencia_media(array): Frecuencia pesada en los bins
    """
    # Definicion de los bins
    if log:
        bin_edges = np.logspace(np.log10(min(lista_x)), np.log10(max(lista_x)), bins)
    else:
         bin_edges = np.linspace(min(lista_x), max(lista_x), bins)

    diccionario = dict() # valor*posicion
    diccionario_frecs = dict()
    for indice_x, value_x in enumerate(lista_x):
        indice_caja = np.searchsorted(bin_edges, value_x)
        if indice_caja in diccionario:
            diccionario[indice_caja].append(lista_x[indice_x]*lista_y[indice_x])
            diccionario_frecs[indice_caja].append(lista_y[indice_x])
        else:
            diccionario[indice_caja] = list()
            diccionario_frecs[indice_caja] = list()
            diccionario[indice_caja].append(lista_x[indice_x]*lista_y[indice_x])
            diccionario_frecs[indice_caja].append(lista_y[indice_x])

    frecuencia_media = []
    for key, value in diccionario_frecs.items():
        if len(value) > 0:
            frecuencia_media.append(np.mean(value))

        elif len(value) == 0:
            frecuencia_media.append(0)

    posiciones = []
    for key, value in diccionario.items():
        if len(value) > 0:
            posiciones.append(np.sum(value)/np.sum(diccionario_frecs[key]))
        elif len(value) == 0:
            posiciones.append((bin_edges[key] + bin_edges[key+1])/2)
            
    ## Ordenamos los datos
    import pandas as pd
    df = pd.DataFrame({"pos": posiciones, "freqs": frecuencia_media})
    df = df.sort_values("pos")

    posiciones = df.pos.values
    frecuencia_media = df.freqs.values
    
    return posiciones, frecuencia_media

In [ ]:
def add_subplot_axes(ax, rect, fig, axisbg='w'):
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3]  # <= Typo was here
    subax = fig.add_axes([x,y,width,height],axisbg=axisbg, xscale = 'log', yscale ='log')
    #subax.xaxis.tick_top()
    subax.xaxis.set_label_position('bottom') 
    #subax.yaxis.tick_right()
    subax.yaxis.set_label_position('left')
    x_labelsize = subax.get_xticklabels()[0].get_size()
    y_labelsize = subax.get_yticklabels()[0].get_size()
    x_labelsize *= rect[2]**0.5
    y_labelsize *= rect[3]**0.5
    subax.xaxis.set_tick_params(labelsize=x_labelsize)
    subax.yaxis.set_tick_params(labelsize=y_labelsize)
    return(subax)

In [ ]:
# Comment for debugging
# import warnings
# warnings.filterwarnings('ignore')

# Linguistic laws in speech: the case of Catalan and Spanish
REQUISITO: Tener una tabla con las duraciones de cada fonema, palabra, etc


# 1. Cargar datos

In [ ]:
input_path = "TablaTranscripcionGlissandoCat.csv"
output_path = ""

# Cargar datos
tabla_datos = pd.read_csv(input_path, delimiter=",", skiprows=1, 
                            names=['token', 'tinit', 'tend', 'duration', 'sentence', 'path', 'tipe', 'numtoken', 'numphonemes','numletters'])
# Para recuperar los okeys
#tabla_datos[tabla_datos.numtoken.isin(tabla_datos[tabla_datos.token == "okay"].numtoken)]
tabla_datos.tail(10)

In [ ]:
# Filtramos y nos quedamos solo con palabras
# No utilizamos News porque repiten el mismo dialogo
lista_words = tabla_datos[((tabla_datos.tipe == 'w') & (~tabla_datos.path.str.contains("News"))
                           & (tabla_datos.token != 'pause') & (tabla_datos.token != "CPsil")
                           & (tabla_datos.token != "CPRCsp"))]

#lista_words.path.str.contains("Turns"))

# Creamos una columna que indique el numero de caracteres de esa palabra
lista_words = lista_words.assign(dur_characteres=lista_words.token.str.len())

In [ ]:
# Filtramos y nos quedamos solo con fonemas
lista_fonema = tabla_datos[((tabla_datos.tipe == 'p') & (~tabla_datos.path.str.contains("News"))) ]

In [ ]:
# Numero de fonemas 
numero= lista_fonema.groupby("token").count().tinit 
len(numero[numero>40])

In [ ]:
# Obtenemos las duraciones de las frases
dur_sentences = tabla_datos[tabla_datos.tipe == "p"].groupby("sentence").sum().duration
# print(lista_fonema.token.unique())
# print(len(lista_fonema.token.unique()))
# tabla_datos[tabla_datos.token == "the"].head()
# tabla_datos[tabla_datos.numtoken == 131]

# Obtener las tablas 1 y 2


In [ ]:
from statistics import mode

def imprimir_estadistica(listaduration, rnd, string):
    
    phonN = len(listaduration)
    phonMean = np.round(np.mean(listaduration), rnd)
    phonStd = np.round(np.std(listaduration), rnd)
    phonMode = mode(np.round(listaduration, rnd))
    phonMedian = np.round(np.median(listaduration), rnd)
    phonP10 = np.round(np.percentile(listaduration, 10), rnd)
    phonP90 = np.round(np.percentile(listaduration, 90), rnd)
    file.write(string + str(phonN) +"  "+ str(phonMean) + "  " + str(phonStd) +
      "  " + str(phonMode) + "  " + str(phonMedian) + "  " + str(phonP10) +
     "  " + str(phonP90))
    file.write("\n")


file = open(output_path + "0_Time_durations.txt","w") 
file.write("                      Time Duration (s)\n")
file.write("                n     Mean  Std  Mode  Median  p10  p90\n")

imprimir_estadistica(lista_fonema.duration, 2, "Phoneme:   ")
imprimir_estadistica(lista_words.duration, 2, "Word:      ")
imprimir_estadistica(dur_sentences, 1, "BG:          ")

file.close()


In [ ]:
def imprimir_estadistica(listaduration, rnd, string):
    
    phonMean = np.round(np.mean(listaduration), rnd)
    phonStd = np.round(np.std(listaduration), rnd)
    phonMode = mode(np.round(listaduration, rnd))
    phonMedian = np.round(np.median(listaduration), rnd)
    phonP10 = np.round(np.percentile(listaduration, 10), rnd)
    phonP90 = np.round(np.percentile(listaduration, 90), rnd)
    file.write(string + str(phonMean) + "  " + str(phonStd) +
      "  " + str(phonMode) + "  " + str(phonMedian) + "  " + str(phonP10) +
     "  " + str(phonP90))
    file.write("\n")


    

file = open(output_path + "0_chars_phons_words_durations.txt","w") 
file.write("                  Num Characters\n")
file.write("             n     Mean  Std  Mode  Median  p10  p90\n")    

numcharperphoneme = lista_words.numletters/lista_words.numphonemes
numcharperphoneme = numcharperphoneme[numcharperphoneme<7]

imprimir_estadistica(numcharperphoneme, 1, "Phoneme:   ")
imprimir_estadistica(lista_words.numletters, 0, "Word:      ")
char_sentences = tabla_datos[tabla_datos.tipe == "w"].groupby("sentence").sum().numletters
imprimir_estadistica(char_sentences, 0, "BG:       ")

file.write("\n")
file.write("                    Num Phonemas\n")
imprimir_estadistica(lista_words.numphonemes, 1, "Word:      ")
phon_sentences = tabla_datos[tabla_datos.tipe == "w"].groupby("sentence").sum().numphonemes
imprimir_estadistica(phon_sentences, 0, "BG:      ")

file.write("\n")
file.write("                     Num Words\n")
words_sentences = tabla_datos[tabla_datos.tipe == "w"].groupby("sentence").count().numphonemes
imprimir_estadistica(words_sentences, 0, "BG:      ")

file.close()

In [ ]:
# Definimos adecuadamente el subconjunto de fonemas y palabras
fonemes_durationsall = lista_fonema.duration.values[(lista_fonema.duration.values<2) & (lista_fonema.duration.values>=0.003)]
words_durationsall = lista_words.duration.values[(lista_words.duration.values<5) & (lista_words.duration.values>=0.003) ]
dur_sentencesall = tabla_datos[tabla_datos.tipe == "w"].groupby("sentence").sum().duration

listainformant = ["s01/", "s02/", "s03/", "s04/", "s05/", "s06/", "s07/", "s08/", "s09/"]

for i in range(1):
    if i == 0:
        fonemes_durations = fonemes_durationsall.round(2)
        words_durations = words_durationsall
        dur_sentences = dur_sentencesall

    else:
        lista_fonema2 = lista_fonema[lista_fonema.path.str.startswith(listainformant[i-1])]
        fonemes_durations2 = lista_fonema2.duration.values[(lista_fonema2.duration.values<5) & (lista_fonema2.duration.values>0)]

        lista_words2 = lista_words[lista_words.path.str.startswith(listainformant[i-1])]
        words_durations2 = lista_words2.duration.values[(lista_words2.duration.values<5) & (lista_words2.duration.values>0) ]

        tabla_datos2 = tabla_datos[tabla_datos.path.str.startswith(listainformant[i-1])]
        dur_sentences2 = tabla_datos2[tabla_datos2.tipe == "w"].groupby("sentence").sum().duration
                       
        fonemes_durations = fonemes_durations2
        words_durations = words_durations2
        #dur_sentences = dur_sentences2


    # Realizamos los histogramas
#     import collections
#     c = collections.Counter(fonemes_durations)
#     c = dict(sorted(c.items()))
#     bin_mean_x_fon = np.asarray(list(c.keys()))
#     counts_norm_fon = np.asarray(list(c.values()))
#     masa_total = (counts_norm_fon*0.01).sum()
#     counts_norm_fon=counts_norm_fon/masa_total
    
    
    bin_mean_x_fon, _, counts_norm_fon, _ = histograma(fonemes_durations, numero_bins=10)
    bin_mean_x_word, _, counts_norm_word, _ = histograma(words_durations, numero_bins=13)
    bin_mean_x_sentence, _, counts_norm_sentence, _ = histograma(dur_sentences, numero_bins=12)


    # Ploteamos
    f, ax = plt.subplots()
    ax.plot(bin_mean_x_fon, counts_norm_fon, '-s', lw = 2, ms = 7, label= 'phonemes', zorder=1, c="darkorange")
    ax.plot(bin_mean_x_word, counts_norm_word, '-o', lw = 2, ms = 7, label= 'words', zorder=2, c='darkblue')
    ax.plot(bin_mean_x_sentence, counts_norm_sentence, '-d', lw = 2, ms = 7, label= 'BG', zorder=3, c="g")


    #ax.set_ylim([-0.3, 11.3])
    ax.set_xlim([0, 1.5])

    ax.set_xlabel("t (seconds)", fontsize=14)
    ax.set_ylabel("P(t)", fontsize=14)

    # Table


    col_labels=[r'$<t>$',r'$mode$']
    row_labels=['phon','words','senten']
    table_vals=[[np.round(np.mean(lista_fonema.duration.values[lista_fonema.duration.values<4]),3),
                 scipy.stats.mode(np.round(lista_fonema.duration.values[lista_fonema.duration.values<4], 2))[0][0]],

                [np.round(np.mean(lista_words.duration.values[lista_words.duration.values<4]),2),
                 scipy.stats.mode(np.round(lista_words.duration.values[lista_words.duration.values<4], 2))[0][0]],

               [np.round(np.mean(dur_sentences), 1),
                   scipy.stats.mode(np.round(dur_sentences, 2))[0][0]]]


    ######################################
    ## COLAPSO DE LOS DATOS##############3
    # Vamos a intentar colapsar los datos
    log_fonemes_dur = np.log(fonemes_durations)
    log_fonemes_dur = (log_fonemes_dur - np.mean(log_fonemes_dur))/np.std(log_fonemes_dur)

    log_words_dur = np.log(words_durations)
    log_words_dur = (log_words_dur - np.mean(log_words_dur))/np.std(log_words_dur)

    log_sentences_dur = np.log(dur_sentences)
    log_sentences_dur = (log_sentences_dur - np.mean(log_sentences_dur))/np.std(log_sentences_dur)

    # Datos Normales 0,1
    randomNormal = np.random.normal(0, 1, 10000000)

    # Realizamos los histogramas
    bin_mean_x_fon_collapsed, _, counts_norm_fon_collapsed, _ = histograma(log_fonemes_dur, numero_bins=10, type_bins="lin")
    bin_mean_x_word_collapsed, _, counts_norm_word_collapsed, _ = histograma(log_words_dur, numero_bins=14, type_bins="lin")
    bin_mean_x_sentence_collapsed, _, counts_norm_sentence_collapsed, _ = histograma(log_sentences_dur, numero_bins=10, type_bins="lin")

    bin_mean_x_normal, _, counts_norm_normal, _ = histograma(randomNormal, numero_bins=200, type_bins="lin")


    # Ploteamos
    subax = plt.axes([0.55, 0.53, .35, .35])
    subax.plot(bin_mean_x_fon_collapsed, counts_norm_fon_collapsed, 's', lw = 2, ms = 7, label= 'phonemes', zorder=1, c="darkorange")
    subax.plot(bin_mean_x_word_collapsed, counts_norm_word_collapsed, 'o', lw = 2, ms = 7, label= 'words', zorder=2, c='darkblue')
    subax.plot(bin_mean_x_sentence_collapsed, counts_norm_sentence_collapsed, 'd', lw = 2, ms = 7, label= 'BG', zorder=3, c="g")
    subax.plot(bin_mean_x_normal, counts_norm_normal, '-s', lw = 1, ms = 0, label= 'phonemes', zorder=5, c = 'k')


    subax.set_xlim([-5.1,5.1])
    #subax.set_ylim([0, 0.53])
    subax.set_xticks([-3,0,3])
    subax.set_yticks([0, 0.2, 0.4])

    subax.set_xlabel("t'", fontsize=14)


    subax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='major', labelsize=14)

    ax.set_xticks([0, 0.5, 1, 1.5])
    #ax.set_yticks([0, 8, 16])

    ax.legend(loc=(0.11, 0.61), frameon=False, fontsize = 12, title=r"$Catalan$", title_fontsize=12)


    if i == 0:
        f.savefig(output_path + "1_Probability_distribution_duration.pdf")
#         ax.set_yscale("log")
#         ax.set_ylim([0.001, 13])
#         ax.set_xlim([0, 5])
#         ax.set_xticks([0, 2, 4])

#         f.savefig("SI_LOG_1_Probability_distribution_duration.pdf")

    else:        
        f.savefig("1_Probability_distribution_duration_informant"+str(i)+".pdf", bbox_inches='tight')

        
##########################################
# Realizamos los histogramas
fonemes_durations = lista_fonema.duration.values[(lista_fonema.duration.values<5) & (lista_fonema.duration.values>=0.03)]
words_durations = lista_words.duration.values[(lista_words.duration.values<5) & (lista_words.duration.values>=0.03) ]
dur_sentences = tabla_datos[tabla_datos.tipe == "w"].groupby("sentence").sum().duration

bin_mean_x_fon, _, counts_norm_fon, _ = histograma(fonemes_durationsall, numero_bins=35)
bin_mean_x_word, _, counts_norm_word, _ = histograma(words_durationsall, numero_bins=15)
bin_mean_x_sentence, _, counts_norm_sentence, _ = histograma(dur_sentencesall, numero_bins=22)

# 3. Ley de Zipf
Calculamos la ley de zip (corpus escrito) para el articulo

In [ ]:
lista_fonema_token_process = lista_fonema.token.str.split(";", expand=True)[0]
lista_fonema_token_process = lista_fonema_token_process.str.split("+", expand=True)[0]


freqs_words = lista_words.token.value_counts()
freqs_phonemes = lista_fonema_token_process.value_counts()

freqs_words.index = np.arange(1, len(freqs_words) + 1)
freqs_words.to_csv(output_path+ "zipf_data.csv")

freqs_phonemes.index = np.arange(1, len(freqs_phonemes) + 1)



f, ax = plt.subplots()
ax.plot(freqs_words[0:30000], 'o', lw = 2, ms = 8, label= 'words', zorder=2, alpha=1, c='darkblue')
ax.plot(freqs_phonemes[0:35], 's', lw = 2, ms = 8, label= 'phonemes', zorder=1,alpha=1, c='darkorange')


# # PENDIENTE PHONEMAS
xphoneme = freqs_phonemes.index.values
yplotphon = xphoneme**(-0.03759105) * 0.9061079**xphoneme
yplotphon = yplotphon/yplotphon[0]*freqs_phonemes.values[0]
ax.plot(xphoneme, yplotphon, "--", color="k", lw = 2)
ax.text(0.4, 0.93, "$Yule(0.04, 0.90)$", horizontalalignment='center', fontsize = 12, verticalalignment='center', transform=ax.transAxes)

# WORDS
# Line pendiente
xline1 = np.array([100, 5000])
yline1 = np.power(1.5*10**-4*xline1, -1.42)

ax.plot(xline1, yline1 , "--", color = "black", lw = 2)
ax.text(0.8, 0.4, r'$\alpha = 1.42$', horizontalalignment='center', fontsize = 12, verticalalignment='center', transform=ax.transAxes)


ax.set_yscale("log")
ax.set_xscale("log")

ax.set_xlabel(r"$rank$",fontsize=14)
ax.set_ylabel(r"$Freq(r)$", fontsize=14)
ax.legend(fontsize=13, frameon=False, loc=3, title=r"$Catalan$", title_fontsize=12)
ax.set_ylim([0.3, 1*10**5])

f.savefig(output_path + "2_Zipf_law.pdf")

# 3. Heaps Law
Para heap law, leemos secuencialmente las conversaciones para ver como aumenta el vocabulario en funcion del tiempo transcurrido o del numero de palabras transcurridas




## RANDOM VERSION

In [ ]:
from scipy.optimize import curve_fit

def funcionlin(x, A, B): # this is your 'straight line' y=f(x)
    return A*x + B
##########################33

import random
random.seed(10)
lista_words["informant"] = lista_words.path.str[0:25]

lista_pendientes=[]


# FIGURA HEAPS RANDOM
f, ax = plt.subplots()
ax2 = ax.twiny()


for i in range(10):
    groups = [df for _, df in lista_words.groupby("informant")]
    random.shuffle(groups)
    lista_word_rand = pd.concat(groups).reset_index(drop=True)
    
    
    # Primero añadimos una columna de zeros
    lista_word_rand['first_time_token'] = 0 
    first_ocurrences_rand =  lista_word_rand.drop_duplicates(subset=["token"]).index
    lista_word_rand.loc[first_ocurrences_rand, "first_time_token"] = 1


    # Creamos el fichero de tiempos
    total_time = 0
    lista_tiempo_transcurrido = []
    path = list(lista_word_rand[0:1].path)[0]

    for index, row in lista_word_rand.iterrows():
        if path != row.path:
            total_time += previoustime # Guarda
            path = row.path
        previoustime = row.tinit
        lista_tiempo_transcurrido.append(total_time+row.tinit)

    xheapt_rand = lista_tiempo_transcurrido
    yheapt_rand = lista_word_rand.first_time_token.cumsum()

    #######SLOPE CALCULATION ##################
    popt, pcov = curve_fit(funcionlin, np.log(xheapt_rand[100::]),np.log(yheapt_rand[100::])) # 
    lista_pendientes.append(popt[0])
    #########################################################
    
    # decimate logaritmic
    decimation = np.logspace(0, np.log10(len(xheapt_rand)-1),100, dtype="int")
    decimation = np.asarray([0] + list(decimation))

    xheapt_rand = np.asarray(xheapt_rand) - xheapt_rand[1] - 0.2
    xheaptdecimate_rand = pd.DataFrame(xheapt_rand).loc[decimation, :]
    xheaptdecimate_rand = xheaptdecimate_rand[0].values

    yheaptdecimate_rand = pd.DataFrame(list(yheapt_rand)).loc[decimation, :]
    yheaptdecimate_rand = yheaptdecimate_rand[0].values

    xheapL_rand = pd.DataFrame(lista_word_rand.index.values + 1).loc[decimation, :]
    xheapL_rand = xheapL_rand[0].values
    
    # Heaps corpus escrito
    yheapL_rand = pd.DataFrame(list(yheapt_rand)).loc[decimation, :]
    yheapL_rand = yheapL_rand[0].values
    
    
    # PLOT
    if i == 0:
        lns1 = ax.plot(2*xheaptdecimate_rand, yheaptdecimate_rand -2 , '-o', lw = 1, ms = 4, label= 'V(T)', zorder=2, alpha=0.6, color="darkblue")
        lns2 = ax2.plot(xheapL_rand, yheaptdecimate_rand, '-d', lw = 1, ms = 4, label= 'V(L)', zorder=5, alpha=0.6, color = "darkolivegreen")
    # PLOT
    if i != 0:
        lns10 = ax.plot(2*xheaptdecimate_rand, yheaptdecimate_rand -2 , '-o', lw = 1, ms = 4, zorder=2, alpha=0.6, color="darkblue")
        lns20 = ax2.plot(xheapL_rand, yheaptdecimate_rand, '-d', lw = 1, ms = 4, zorder=5, alpha=0.6, color = "darkolivegreen")
    
    
xline = np.array([2*10**2, 5*10**4])
ax.text(0.82, 0.57, r'$\beta = \gamma =$' + str(np.mean(lista_pendientes).round(2)), horizontalalignment='center', fontsize = 14, verticalalignment='center', transform=ax.transAxes)
ax.plot(xline, 2* np.power(xline, np.mean(lista_pendientes)), "--", color = "k", lw=2)

ax.set_xlim([5*10**-1, 2*10**5])

ax.set_xlabel(r"$Time$" +" " + r"$elapsed$" +" " + r"$T$"+ " " + r"$(seconds)$", fontsize=14)
ax.set_ylabel( r"$Vocabulary$" + " " +  r"$V$", fontsize=14)

ax.set_xscale("log")
ax.set_yscale("log")

ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.set_xlim([5*10**-1, 2*10**5])


ax2.set_xlabel(r"$Words$" +" " + r"$elapsed$" + " " + r"$L$", fontsize=14)


# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=2, frameon = False, fontsize = 14, title=r"$Catalan$", title_fontsize=12)


#ax.set_xticks([10**0,10**2, 10**4, 10**4])
#ax2.set_xticks([10**0,10**2, 10**4, 10**5])
#ax.set_yticks([10**0,10**2, 10**4])

ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax2.tick_params(axis='x', labelsize=12)


f.savefig(output_path + "4_HeapLaw_RANDOM_order.pdf")
    
    
    

# 4. Brevity law 
Calculamos la ley de brevedad chequeando que las palabras más recuentes tienden a ser más cortas
Entendemos tres posibles definiciones:


## 4.1 WORDS
1. Frequencia palabras vs duracion en segundos
2. Frecuencia palabras vs numero de fonemas
3. Frecuencia de palabras vs numero de caracteres

In [ ]:
# Creamos una columna que indique las veces que ha ocurrido esa palabra en nuestra base de datos
lista_words = lista_words.assign(repetitions=lista_words.token.map(lista_words.token.value_counts()))


In [ ]:
from scipy.optimize import curve_fit

def func_exp(x, a, b):
    return a * np.exp(-b * x)

def func_log(x, a, b): # FUNCION LOGARITMICA que utiliza los mismos parametros
    return 1/(-b) * ( np.log(x) - np.log(a) )




## 1. Frequencia palabras vs duracion segundos
#brevity0 = lista_words.groupby("repetitions").duration.mean()
brevity0 = lista_words.groupby("token").median()[["duration", "repetitions"]]


#The two-sided p-value for a hypothesis test whose null hypothesis is that two sets of data are uncorrelated, 
#A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis
print("Frequencia palabras vs duracion(segundos:")
print(scipy.stats.spearmanr(brevity0.duration, np.log(brevity0.repetitions)))

f, ax = plt.subplots()


### AHORA VAMOS A HACER UN BINING
freq0, pos0  = binear_datos(brevity0.repetitions, brevity0.duration, bins=11, log = True)

ax.plot(brevity0.duration, brevity0.repetitions, 'o', lw = 1, ms = 1, zorder=2, alpha=0.6, color = 'lightgray',fillstyle= "none")
ax.plot(pos0, freq0, 'o', lw = 2, ms = 10, label= 'words', zorder=2, alpha=1, color="darkblue")

ax.set_xlabel(r"$Median$"+ " " + r"$duration$"+ " " + r"$(seconds)$", fontsize=13)
ax.set_ylabel("Frequency", fontsize=13)
ax.set_yscale("log")
#ax.set_xscale("log")
ax.set_xlim([0,0.7])
ax.set_ylim([0.5,1*10**4])



## FITING
x = brevity0.repetitions
y = brevity0.duration

xplot = np.linspace(0.1, 0.55, 8)
popt, pcov = curve_fit(func_log, x, y)
ax.plot(xplot, func_exp(xplot, *popt), "--", color = "darkred", lw = 2)

ax.text(0.75, 0.25, r'$Catalan$' + "\n" + r'$\lambda= $' + "{:.{}f}".format(popt[1], 1), horizontalalignment='center', fontsize = 15, verticalalignment='center', transform=ax.transAxes)




residuals = y - func_log(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R2: " + str(r_squared))
print()


##############################################
## 2.frecuencia palabras vs numero de fonemas
##############################################
def func_exp64(x, a, b):
    return a * 35**(-b * x)

def func_log64(x, a, b): # FUNCION LOGARITMICA que utiliza los mismos parametros
    return 1/(-b) * ( np.log(x)/np.log(35) - np.log(a)/np.log(35) )




brevity1 = lista_words.groupby("token").median()[["numphonemes", "repetitions"]]

print("Frequencia palabras vs numero de fonemas:")
print(scipy.stats.spearmanr(brevity1.numphonemes, np.log(brevity1.repetitions)))


# Ploteamos
subax = plt.axes([ax.get_position().x1 - .28, ax.get_position().y1 - .28, .28, .28])

freq1, pos1 = binear_datos(brevity1.repetitions, brevity1.numphonemes, bins=11, log = True)
subax.plot(brevity1.numphonemes, brevity1.repetitions, 'o', lw = 1, ms = 2, zorder=2, alpha=0.6, color = 'lightgray', fillstyle= "none")
subax.plot(pos1, freq1, 'o', lw = 2, ms = 7, label= 'words', zorder=2, alpha=1, color="darkblue")



## FITING#######################################################
x = brevity1.repetitions
y = brevity1.numphonemes

xplot = np.linspace(1.5, 8, 3)
popt, pcov = curve_fit(func_log64, x, y)
subax.plot(xplot, func_exp64(xplot, *popt), "--", color = "darkred", lw = 2)
subax.text(0.74, 0.73, r'$\lambda_D = $' + "{:.{}f}".format(popt[1], 2), horizontalalignment='center', fontsize = 12, verticalalignment='center', transform=ax.transAxes)




residuals = y - func_log64(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R2: " + str(r_squared))
print()


subax.set_yscale("log")
subax.set_xlabel(r"$\widetilde{size}$" + " " +  r"$(phonemes)$", fontsize=12)
subax.set_xlim([0.5, 7])
subax.set_ylim([0.3, 50000])
#subax.set_ylabel("freq",rotation=0)


##############################################
## 3.frecuencia palabras vs numero de characters
##############################################
def func_exp26(x, a, b):
    return a * 26**(-b * x)

def func_log26(x, a, b): # FUNCION LOGARITMICA que utiliza los mismos parametros
    return 1/(-b) * ( np.log(x)/np.log(26) - np.log(a)/np.log(26) )




brevity2 = lista_words.groupby("token").median()[["numletters", "repetitions"]]

brevity2=brevity2[brevity2.numletters<15]
print("Frequencia palabras vs numero de caracteres:")
print(scipy.stats.spearmanr(brevity2.numletters, np.log(brevity2.repetitions)))


# Ploteamos
subax2 = plt.axes([ax.get_position().x0, ax.get_position().y0, .28, .28])

freq2, pos2 = binear_datos(brevity2.repetitions, brevity2.numletters, bins=7, log = True)
subax2.plot(brevity2.numletters, brevity2.repetitions, 'o', lw = 1, ms = 2, zorder=2, alpha=0.4, color = 'lightgrey')
subax2.plot(pos2, freq2, 'o', lw = 2, ms = 7, label= 'words', zorder=2, alpha=1, color="darkblue")


## FITING#######################################################
x = brevity2.repetitions
y = brevity2.numletters

xplot = np.linspace(2, 12, 3)
popt, pcov = curve_fit(func_log26, x, y)
subax2.plot(xplot, func_exp26(xplot, *popt), "--", color = "darkred", lw = 2)
subax2.text(0.12, 0.12, r'$\lambda_D = $' + "{:.{}f}".format(popt[1], 2), horizontalalignment='center', fontsize = 12, verticalalignment='center', transform=ax.transAxes)



residuals = y - func_log26(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R2: " + str(r_squared))



subax2.set_yscale("log")
#subax2.set_xscale("log")

subax2.set_xlabel(r"$size$" + " " +  r"$(chars)$",rotation=0, fontsize=12)
subax2.set_xlim([0.5, 8])
subax2.set_ylim([0.2, 10**5])

subax2.yaxis.set_label_position("right")
subax2.yaxis.set_ticks_position("right")

#subax2.set_ylabel("freq",rotation=0)
subax2.xaxis.set_label_position("top")
subax2.xaxis.set_ticks_position("top")



subax2.set_xticks([1,3,5,7])
subax2.set_yticks([10**0,10**2, 10**4])

#subax2.set_yticks([10**3,10**5])
ax.set_xticks([0.1, 0.3, 0.5, 0.7])


ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

f.savefig(output_path + "3_Brevity_words.pdf")



# 4.2 Brevity law  Phonemes

In [ ]:
tabla_datos_brev = tabla_datos.copy()
#tabla_datos_brev = tabla_datos_brev.assign(numletterPhon=tabla_datos_brev[tabla_datos_brev.tipe=="w"].numletters/tabla_datos_brev[tabla_datos_brev.tipe=="w"].numphonemes)
#tabla_datos_brev.fillna(method='ffill', inplace=True)

lista_fonemabrev = tabla_datos_brev[((tabla_datos_brev.tipe == 'p') & (tabla_datos_brev.token != 'SIL') & (tabla_datos_brev.token != 'VOCNOISE') 
                            & (tabla_datos_brev.token != 'UNKNOWN') & (tabla_datos_brev.token != 'NOISE') & (tabla_datos_brev.token != 'LAUGH')
                            & (tabla_datos_brev.token != 'IVER')
                            & (tabla_datos_brev.token.str.endswith("}") == False) & (tabla_datos_brev.token.str.startswith("{") == False))]

lista_fonemabrev = lista_fonemabrev.assign(repetitions=lista_fonemabrev.token.map(lista_fonemabrev.token.value_counts()))

In [ ]:
brevityp = lista_fonemabrev.groupby("token").median()[["duration", "repetitions"]]
brevityp = brevityp[brevityp.repetitions>50]
brevityp2 = brevityp[brevityp.repetitions>80]

print(len(brevityp))
#The two-sided p-value for a hypothesis test whose null hypothesis is that two sets of data are uncorrelated, 
#A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis
print("Frequencia phonemes vs duracion(segundos:")
print(scipy.stats.spearmanr(brevityp.repetitions, brevityp.duration))
print()
print()

print("Frequencia phonemes vs duracion(segundos) si >50:")
print(scipy.stats.spearmanr(brevityp2.repetitions, brevityp2.duration))
print()
print()

### AHORA VAMOS A HACER UN BINING
freq, pos = binear_datos(brevityp.repetitions, brevityp.duration, bins=5, log = True)

f, ax1 = plt.subplots()
ax1.plot(brevityp.duration, brevityp.repetitions, 'o', lw = 1, ms = 5, zorder=2, alpha=0.6, color = 'lightgrey')
ax1.plot(pos, freq, 's', lw = 2, ms = 11, label= 'phonemes', zorder=2, alpha=1, color = 'darkorange')

ax1.set_xlabel(r"$Median$"  + " " + r"$time$"  + " " + r"$ duration$"  + " " + r"$ (seconds)$", fontsize=12)
#ax.legend()
ax1.set_yscale("log")

ax1.set_ylabel("Frequency", fontsize=12)
#ax1.legend(frameon=False)

ax1.set_xticks([0.05, 0.07, 0.13])


#######################################################################################
def func_exp(x, a, b):
    return a * np.exp(-b * x)

def func_log(x, a, b): # FUNCION LOGARITMICA que utiliza los mismos parametros
    return 1/(-b) * ( np.log(x) - np.log(a) )

## FITING
x = brevityp2.repetitions
y = brevityp2.duration

xplot = np.linspace(0.05, 0.07, 3)
popt, pcov = curve_fit(func_log, x, y)
ax1.plot(xplot, func_exp(xplot, *popt), "--", color = "darkred", lw = 2)

ax1.text(0.8, 0.8, r'$Catalan$' + "\n" + r'$\lambda = $' + "{:.{}f}".format(popt[1], 0), horizontalalignment='center', fontsize = 15, verticalalignment='center', transform=ax.transAxes)

residuals = y - func_log(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R2: " + str(r_squared))
print()

ax1.set_xlim([0.04, 0.09])

ax1.tick_params(axis='x', labelsize=12)
ax1.tick_params(axis='y', labelsize=12)

f.savefig(output_path + "3_Brevity_phonemes.pdf")


# Size-rank brevity law

In [ ]:
meanDuration = lista_words.groupby("token").duration.median()
freq = lista_words.groupby("token").repetitions.mean()
df = pd.DataFrame({"dur":meanDuration, "f":freq })
df = df.sort_values("f", ascending=False)
df["r"] = np.arange(1,len(df)+1)

dfRsup50 = df[df.r>0].copy()

df["cumdur_rangos"] = df.dur.cumsum()
dfRsup50["cumdur_rangos"] = dfRsup50.dur.cumsum()



###########################################################
# PLOT ####################################################

f, ax = plt.subplots()

meanx, meany = binear_datos(dfRsup50.r, dfRsup50.dur, bins=21, log = True)
ax.plot(dfRsup50.r, dfRsup50.dur, "o", color = "lightgray", alpha = 0.8, ms=1, label =r"$data$")

ax.plot(meanx[0:-1], meany[0:-1], "o", color = "darkblue", ms=9, alpha = 0.8, label =r"$binned \  data$")



# Primera pendiente
xplot = dfRsup50.r
yplot = 0.059663866*np.log(meanx[2:-1])
yplot = yplot - yplot[0] + meany[0]

ax.plot(meanx[2:-1], yplot - 0.06, "--", color="k", lw=2, label = r"$\ell \sim \frac{\alpha}{\lambda}\log(r)$") #label =r"$\hat{\ell}_1 = \frac{\alpha _1}{\lambda} \cdot \log(r _i) + K_1$")



ax.set_xscale("log")

ax.set_xlabel("Rank r ", fontsize=14)
ax.set_ylabel(r"$\hat{\ell} $", fontsize=14)

ax.set_yticks([0.2, 0.5, 0.8])

ax.set_xlim([0.8, 6000])

ax.set_ylim([0.01, 0.8])
ax.tick_params(axis='both', which='major', labelsize=14)

ax.legend(frameon=False, fontsize=12,title=r"$Catalan$", title_fontsize=12)
f.savefig(output_path + "SI_RANK_DURATION.pdf")



# 6. Menzerath Altmann law

 Existen divesas posibles definiciones de Menzerath Altmann law. Por ello vamos a intentar varias y comaprarlas

## 6.1 Sentences length (number of words) VS words length (number of letters)

In [ ]:
SentenceSize_numwords = []
WordsSize_numletters = []
WordsSize_numphonemes = []  # Para la definicion2
WordsSize_seconds = []  # Para la definicion3



for sentence in tabla_datos.sentence.unique():
    tablaSentence = tabla_datos[tabla_datos.sentence == sentence]
    numwords = len(tablaSentence[tablaSentence.tipe == "w"])
    if numwords == 100:
        stop
    SentenceSize_numwords.append(numwords)

    numletters_mean = np.mean(tablaSentence[tablaSentence.tipe == "w"].numletters)
    WordsSize_numletters.append(numletters_mean)
    
    # Para la definicion2
    numphonemes_mean = np.mean(tablaSentence[tablaSentence.tipe == "w"].numphonemes)
    WordsSize_numphonemes.append(numphonemes_mean)
    
    # Para la definicion3
    word_duration_mean = np.mean(tablaSentence[tablaSentence.tipe == "w"].duration)
    WordsSize_seconds.append(word_duration_mean)

    
    


In [ ]:
from scipy.optimize import curve_fit
def func_powerlawexp(x, alfa, beta, c):
    return alfa * x**(beta) * np.exp(-c*x)

# def func_powerlawexp(x, beta):
#     #c = b/6
#     #0.4 = a*e^(-b/6)
    
#     return (0.4/(np.exp(-beta/6)) * x**(-beta) * np.exp(-beta/6*x))

numbins=10

for setsize in [41]: # [41, 80]
    for bin15 in [False]:
    
        # FIGURA MENZERATH
        f, ax = plt.subplots()

        ###########################################################################
        # MENZERATH FRSES WORDS (SECONDS) #########################################
        #########################################################################################################

        Mentable_sec = pd.DataFrame({"SentSize":SentenceSize_numwords, "WordsSize": WordsSize_seconds})
        Mentable_sec1 = Mentable_sec[ Mentable_sec["SentSize"]<setsize]

        azul = Mentable_sec1.groupby("SentSize").mean()
        pos0 = azul.index.values
        freq0 = azul.WordsSize.values
        pos015, freq015 = binear_datos(pos0, freq0, bins=numbins, log = False)


        # MAIN PLOT
        #ax.plot(Mentable_sec.SentSize.values, Mentable_sec.WordsSize.values , 'o', lw = 2, ms = 2, zorder=2, alpha=1, color = "gray")
        ax.plot(Mentable_sec1.SentSize.values[0:3000], Mentable_sec1.WordsSize.values[0:3000], 'o', ms = 1, zorder=2, alpha=0.5, color = 'lightgrey', label = r"$data$")
        if bin15 == False:
            ax.plot(pos0, freq0, 'o', lw = 2, ms = 8, zorder=2, alpha=1, color = "darkblue", label = r"$mean$")
        elif bin15 == True:
            ax.plot(pos015, freq015, 'o', lw = 2, ms = 8, zorder=2, alpha=1, color = "darkblue", label = r"$mean$")

        ax.set_xlabel("BG size in number of words")
        ax.set_ylabel(r"$Word$" + " " + r"$\overline{size}$" + " " + r"$(seconds)$")

        ax.set_ylim([0.0, 0.45])

        ax.set_xticks([0, 15, 30, 45])
        ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])


        x = np.asarray(pos0)
        y = np.asarray(freq0)


        # POWER LAW EXPONENTIAL
        print("Power law exponential seconds:")
        #popt, pcov = curve_fit(func_powerlawexp, x, y)
        popt, pcov = curve_fit(func_powerlawexp, pos0, freq0)

        expy = func_powerlawexp(x, popt[0], popt[1], popt[2])
        ax.plot(x, expy, "--", lw=2, color="red", label = r"$fit$" + " " + r"$to$" + " " + r"$y = ax^b e^{-cx}$")

        velocityx = x
        velocityy = 1/expy

        print("exponentes = " + str(popt))

        # GET R2
        residuals = y - func_powerlawexp(x, popt[0], popt[1], popt[2])
        # residuals = y - func_powerlawexp(x, popt[0])

        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        R2 = 1 - (ss_res / ss_tot)
        print("R2:" + str(R2))
        print("")

        ax.legend(frameon = False, loc = 4, fontsize=11,title=r"$Catalan$", title_fontsize=12)

        #ax.set_xlim([0, 40])


        ax.tick_params(axis='x', labelsize=12)
        ax.tick_params(axis='y', labelsize=12)

        # NUM LETTERS ############################################################################
        Mentableletters = pd.DataFrame({"SentSize":SentenceSize_numwords, "WordsSize": WordsSize_numletters})
        Mentableletters1 = Mentableletters[ Mentableletters["SentSize"]<setsize]
        #Menzerathletters = Mentableletters.groupby("SentSize").mean()

        subax = plt.axes([ax.get_position().x0, ax.get_position().y0, .25, .25])

        #pos1, freq1 = binear_datos(Mentableletters1.SentSize.values, Mentableletters1.WordsSize.values, bins=15, log = False)
        azul = Mentableletters1.groupby("SentSize").mean()
        pos1 = azul.index.values
        freq1 = azul.WordsSize.values
        
        pos115, freq115 = binear_datos(pos1, freq1, bins=numbins, log = False)



        # MAIN PLOT
        #subax.plot(Mentableletters.SentSize.values, Mentableletters.WordsSize.values , 'o', lw = 2, ms = 2, zorder=2, alpha=1, color = "gray")
        #subax.plot(Menzerathletters, 'o', lw = 2, ms = 2, label= 'seconds', zorder=2, alpha=1, color = "gray")
        subax.plot(Mentableletters.SentSize.values[0:3000], Mentableletters.WordsSize.values[0:3000], 'o', ms = 1, zorder=2, alpha=0.5, color = 'lightgrey')

        
        if bin15 == False:
            subax.plot(pos1, freq1, 'o', lw = 2, ms = 5, zorder=2, alpha=1, color = "darkblue")
        elif bin15 == True:
            subax.plot(pos115, freq115, 'o', lw = 2, ms = 5, zorder=2, alpha=1, color = "darkblue")
        
        

        #subax.set_xlabel("BG (words)")
        subax.set_ylabel( r"$Word$" + " " + r"$\overline{size}$" + "\n" + r"$(chars)$", rotation = 0)
        subax.yaxis.set_label_coords(1.22, 0.65)


        subax.yaxis.set_label_position("right")
        subax.yaxis.set_ticks_position("right")

        #subax2.set_ylabel("freq",rotation=0)
        subax.xaxis.set_label_position("top")
        subax.xaxis.set_ticks_position("top")
        subax.set_ylim([3.5, 4.1])

        subax.set_yticks([3.6, 4])
        subax.set_xticks([10, 30, 50])
        #subax.set_xlim([0, 40])


        x = np.asarray(pos1)[2::]
        y = np.asarray(freq1)[2::]

        subax.tick_params(axis='x', labelsize=12)
        subax.tick_params(axis='y', labelsize=12)

        # POWER LAW EXPONENTIAL
        print("Power law exponential chars:")
        popt, pcov = curve_fit(func_powerlawexp, x, y)
        expy = func_powerlawexp(x, popt[0], popt[1], popt[2])
        subax.plot(x, expy, "--", lw=2, color="red")
        print("exponentes = " + str(popt))

        # GET R2
        residuals = y - func_powerlawexp(x, popt[0], popt[1], popt[2])
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        R2 = 1 - (ss_res / ss_tot)
        print("R2:" + str(R2))
        print("")


        #################################################################################
        # Sentences length (number of words) VS words length (number of phonemes)
        #############################################################################

        # Ploteamos
        subax2 = plt.axes([ax.get_position().x1 - .25, ax.get_position().y1 - .25, .25, .25])

        Mentable = pd.DataFrame({"SentSize":SentenceSize_numwords, "WordsSize": WordsSize_numphonemes})
        Mentable1 = Mentable[Mentable.SentSize<setsize]
        #Menzerathphonemes = Mentable.groupby("SentSize").mean()


        #pos2, freq2 = binear_datos(Mentable1.SentSize.values, Mentable1.WordsSize.values, bins=15, log = False)
        azul = Mentable1.groupby("SentSize").mean()
        pos2 = azul.index.values
        freq2 = azul.WordsSize.values
        pos215, freq215 = binear_datos(pos2, freq2, bins=numbins, log = False)

        # MAIN PLOT
        #subax2.plot(Menzerath, 'o', lw = 2, ms = 2, zorder=2, alpha=1, color = "gray")
        subax2.plot(Mentable.SentSize.values[0:3000], Mentable.WordsSize.values[0:3000], 'o', ms = 1, zorder=2, alpha=0.5, color = 'lightgrey')
        if bin15 == False:
            subax2.plot(pos2, freq2, 'o', lw = 2, ms = 5, zorder=2, alpha=1, color = "darkblue")
        elif bin15 == True:
            subax2.plot(pos215, freq215, 'o', lw = 2, ms = 5, zorder=2, alpha=1, color = "darkblue")
        
        
        #subax2.set_xlabel(r"$BG (words)$")
        subax2.set_ylabel(r"$Word$" + " " + r"$\overline{size}$" + "\n" + r"$(phon)$", rotation = 0)
        subax2.yaxis.set_label_coords(-0.25, 0.35)

        subax2.set_xticks([10, 30, 50])
        subax2.set_yticks([3.4, 3.8])
        subax2.set_ylim([3.3,3.9])
        #subax2.set_xlim([0, 40])
        subax2.tick_params(axis='x', labelsize=12)
        subax2.tick_params(axis='y', labelsize=12)

        x = np.asarray(pos2)[2::]
        y = np.asarray(freq2)[2::]

        # POWER LAW EXPONENTIAL
        print("Power law exponential phonemes:")
        popt, pcov = curve_fit(func_powerlawexp, x, y)
        expy = func_powerlawexp(x, popt[0], popt[1], popt[2])
        subax2.plot(x, expy, "--", lw=2, color="red")
        print("exponentes = " + str(popt))

        # GET R2
        residuals = y - func_powerlawexp(x, popt[0], popt[1], popt[2])
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        R2 = 1 - (ss_res / ss_tot)
        print("R2:" + str(R2))
        print("")

        if setsize == 80:
            if bin15 == False:
                f.savefig(output_path + "5_Menzerath_BG_WORDS_allBG.pdf")
            elif bin15 == True:
                f.savefig(output_path + "5_Menzerath_BG_WORDS_allBG_bin15.pdf")
        elif setsize == 41:
            subax.set_xticks([10, 30])
            subax2.set_xticks([10, 30])
            subax2.set_xlim([0, 42])
            subax.set_xlim([0, 42])
            
            if bin15 == False:
                f.savefig(output_path + "5_Menzerath_BG_WORDS_less41.pdf")
            elif bin15 == True:
                f.savefig(output_path + "5_Menzerath_BG_WORDS_less41_bin15.pdf")




## Todos los posibles ajustes de MAL

In [ ]:
def func_powerlawexp(x, alfa, beta, c):
    return alfa * x**(beta) * np.exp(-c*x)


def adjust_powerlawexp(xdata, ydata):
    popt, pcov = curve_fit(func_powerlawexp, xdata, ydata)
    expy = func_powerlawexp(xdata, popt[0], popt[1], popt[2])

    # GET R2
    residuals = ydata - func_powerlawexp(xdata, popt[0], popt[1], popt[2])
    # residuals = y - func_powerlawexp(x, popt[0])

    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata - np.mean(ydata))**2)
    R2 = 1 - (ss_res / ss_tot)
    print("a|b|c| b/c = " + str(popt[0].round(2)) + " | " + str(popt[1].round(4)) +" | "+  str(popt[2].round(5))+" | "+  str((popt[1]/popt[2]).round()) + "  R2: " + str(R2.round(2)))
    print("")


# ALL DATA BIN AZUL
print("###### ALL DATA AZUL ############################")
azul = Mentable_sec.groupby("SentSize").mean()
print("BG seconds  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)

Mentable1 = Mentable[Mentable.SentSize>2]
azul = Mentable1.groupby("SentSize").mean()
print("BG>2 phonemes  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)

azul = Mentable.groupby("SentSize").mean()
print("BG phonemes  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)

Mentable1 = Mentableletters[Mentableletters.SentSize>2]
azul = Mentable1.groupby("SentSize").mean()
print("BG>2 letters  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)

azul = Mentableletters.groupby("SentSize").mean()
print("BG letters  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)




# ALL DATA BIN AZUL
print("###### DATA AZUL < 40 ############################")

Mentable_sec1 = Mentable_sec[Mentable_sec.SentSize<40]
azul = Mentable_sec1.groupby("SentSize").mean()
print("BG seconds  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)


Mentable1 = Mentable[Mentable.SentSize<40]
Mentable1 = Mentable1[Mentable1.SentSize>2]
azul = Mentable1.groupby("SentSize").mean()
print("BG>2 phonees  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)

Mentable1 = Mentable[Mentable.SentSize<40]
azul = Mentable1.groupby("SentSize").mean()
print("BG phonees  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)



Mentableletters1 = Mentableletters[Mentableletters.SentSize<40]
Mentableletters1 = Mentableletters1[Mentableletters1.SentSize>2]
azul = Mentableletters1.groupby("SentSize").mean()
print("BG>2 letters  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)


Mentableletters1 = Mentableletters[Mentableletters.SentSize<40]
azul = Mentableletters1.groupby("SentSize").mean()
print("BG letters  ")
adjust_powerlawexp(azul.index.values, azul.WordsSize.values)


# ALL DATA GRIS
print("###### ALL DATA GRIS ############################")
print("BG seconds  ")
adjust_powerlawexp(Mentable_sec.SentSize.values, Mentable_sec.WordsSize.values)

print("BG >2 phonemes  ")
Mentable1 = Mentable[Mentable.SentSize>2]
adjust_powerlawexp(Mentable1.SentSize.values, Mentable1.WordsSize.values)

print("BG phonemes  ")
adjust_powerlawexp(Mentable.SentSize.values, Mentable.WordsSize.values)

print("BG letters >2 ")
Mentable1 = Mentableletters[Mentableletters.SentSize>2]
adjust_powerlawexp(Mentable1.SentSize.values, Mentable1.WordsSize.values)

print("BG letters  ")
adjust_powerlawexp(Mentableletters.SentSize.values, Mentableletters.WordsSize.values)


# DATA < 40 GRIS
print("###### DATA GRIS < 40 ############################")
print("BG seconds  ")
Mentable_sec1 = Mentable_sec[Mentable_sec.SentSize<40]
adjust_powerlawexp(Mentable_sec1.SentSize.values, Mentable_sec1.WordsSize.values)

print("BG>2 phonemes  ")
Mentable1 = Mentable[Mentable.SentSize<40]
Mentable1 = Mentable1[Mentable1.SentSize>2]
adjust_powerlawexp(Mentable1.SentSize.values, Mentable1.WordsSize.values)

print("BG phonemes  ")
Mentable1 = Mentable[Mentable.SentSize<40]
adjust_powerlawexp(Mentable1.SentSize.values, Mentable1.WordsSize.values)

print("BG >2 letters  ")
Mentableletters1 = Mentableletters[Mentableletters.SentSize<40]
Mentableletters1 = Mentableletters1[Mentableletters1.SentSize>2]
adjust_powerlawexp(Mentableletters1.SentSize.values, Mentableletters1.WordsSize.values)


print("BG letters  ")
Mentableletters1 = Mentableletters[Mentableletters.SentSize<40]
adjust_powerlawexp(Mentableletters1.SentSize.values, Mentableletters1.WordsSize.values)


# 5.4 Words length (number of phonemes) VS phonemes length (seconds)


In [ ]:
#WordLength_num_phonemes = tabla_datos[tabla_datos.tipe == "w"].numphonemes
WordLength_num_phonemes = tabla_datos[tabla_datos.tipe == "p"].groupby("numtoken").count()["token"]
PhonemeLength_seconds = tabla_datos[tabla_datos.tipe == "p"].groupby("numtoken").mean().duration

In [ ]:
def func_powerlawexp(x, alfa, beta, gamma):
    return alfa * x**(beta) * np.exp(-gamma*x)

Mentable = pd.DataFrame({"WordSize":WordLength_num_phonemes.values, "PhoneSize": PhonemeLength_seconds.values})
Menzerath = Mentable.groupby("WordSize").mean()
Menzerath = Menzerath[0:17]


# FIGURA MENZERATH
f, ax = plt.subplots()
pos0, freq0 = binear_datos(Mentable.WordSize.values, Mentable.PhoneSize.values, bins=12, log = False)

# MAIN PLOT
ax.plot(Mentable.WordSize.values[0:1000], Mentable.PhoneSize.values[0:1000], 'o', ms = 1, zorder=2, alpha=0.5, label = r"$data$", color = 'lightgrey')
ax.plot(Menzerath, 's', lw = 2, ms = 11, zorder=2, alpha=0.8, color="darkorange", label=r"$mean$")



ax.set_xlabel("Word length(<phonem>)",fontsize=12)
ax.set_ylabel("Phoneme length (<sec>)", fontsize=12)

#ax.set_xscale("log")
#ax.set_yscale("log")


ax.set_xticks([2, 6, 10, 14, 18])
ax.set_yticks([0.05, 0.08, 0.11])
ax.set_ylim([0.03,0.13])
ax.set_xlim([1, 19])

x = pos0
y = freq0

# POWER LAW EXPONENTIAL
print("Power law exponential:")
popt, pcov = curve_fit(func_powerlawexp, x, y)
expy = func_powerlawexp(x, popt[0], popt[1], popt[2])
ax.plot(x, expy, "--", lw=2, color="red", label = r"$fit$" + " " + r"$to$" + " " + r"$y = ax^b e^{-cx}$")
print("exponentes = " + str(popt))

# GET R2
residuals = y - func_powerlawexp(x, popt[0], popt[1], popt[2])
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
R2 = 1 - (ss_res / ss_tot)
print("R2:" + str(R2))
print("")


ax.legend(frameon = False, loc = "best", fontsize=12,title=r"$Catalan$", title_fontsize=12)


ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)


ax.set_xlabel(r"$Word$" + " " + r"$\overline{size}$" + " " + r"$in$" + " " + r"$number$" + " " + r"$of$" + " " + r"$phonemes$")
ax.set_ylabel(r"$Phoneme$" + " " + r"$\overline{size}$" + " " + r"$(seconds)$")


f.savefig(output_path + "5_Menzerath_Words_Phonemes.pdf")

